<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Attention_Mechanisms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attention = nn.Linear(hidden_dim, 1)

    def forward(self, hidden_states):
        scores = self.attention(hidden_states).squeeze(-1)
        attention_weights = torch.softmax(scores, dim=1)
        context_vector = torch.sum(hidden_states * attention_weights.unsqueeze(-1), dim=1)
        return context_vector, attention_weights

hidden_dim = 128
attention = Attention(hidden_dim)  # Removed .to('cuda')

# Dummy data for testing
hidden_states = torch.rand(32, 10, hidden_dim)  # Removed .to('cuda')

context_vector, attention_weights = attention(hidden_states)
print("Context vector shape:", context_vector.shape)
print("Attention weights shape:", attention_weights.shape)